# The HouseBlend example

In [1]:
import os
import sys
import time
import matplotlib.pyplot as plt
import numpy as np

module_path = os.path.abspath(os.path.join('../HouseBlend'))
if module_path not in sys.path:
    sys.path.append(module_path)
import HouseBlend as hb

## Setup an example test case
If using for real, first create the contacts.xlsx workbook following the template and leave the test parameter as False (default value).

### User inputs
**n_periods** <br>
The total number of periods you wish to schedule including historic and future. 
Set this to None to have the minimum number of periods required for everyone to meet everyone else

**current_period** <br>
The period number of the first future period to schedule. 
For first ever usage, this should be set to 1 (i.e. you need to schedule from period 1 forwards).
Values > 1 will import previous periods from schedule.xlsx and treat these as having happened and therefore fixed.

**parent_dir** <br>
The folder path containing the input / output files. If it does not exist, it will be created.

In [2]:
n_periods = None  # total number of periods to schedule. Set to None for minimum number to satisfy all possible meetings.

current_period = 1  # the period from which to schedule. Set to 1 for first usage.

parent_fullpath = os.path.join('../', 'example')  # the folder containing the input files (contacts.xlsx). Outputs will be saved here.

## Import input files or create test participants
Use import_contacts_df with test set to an integer to create a new test group of n participants. For real usage, remove test parameter (default is false) to use on a defined contacts.xlsx input file.

Import historic schedule if it exists - required for historic inclusion e.g. if current_period>1, setting test=True deletes any previous schedule!

In [3]:
contacts, dates, availability = hb.import_contacts_df(folderpath=parent_fullpath, test=6)

schedule, bool_schedule = hb.import_schedules(folderpath=parent_fullpath, test=True)

contacts.head()

Deleting previous contacts file
Creating new contact directory of 6 length for testing
Deleting previous schedule files
No existing schedule found. Ensure current_period is set to 1.


,email,Assistant,Assistant email
Person,,,
Connie Black,ConnieBlack@email.com,Connie Black,ConnieBlack@email.com
Jennifer Rodriguez,JenniferRodriguez@email.com,Jennifer Rodriguez,JenniferRodriguez@email.com
Felicia Vaughn,FeliciaVaughn@email.com,Felicia Vaughn,FeliciaVaughn@email.com
Mark Ware,MarkWare@email.com,Mark Ware,MarkWare@email.com
Anna Gibson,AnnaGibson@email.com,Anna Gibson,AnnaGibson@email.com


In [4]:
dates

,Start Date,End Date
Period,,
1,2025-05-11,2025-05-25
2,2025-05-25,2025-06-08
3,2025-06-08,2025-06-22
4,2025-06-22,2025-07-06
5,2025-07-06,2025-07-20


## Run schedule optimisation

In [11]:
bool_schedule = hb.run_schedule_optimisation(contacts, bool_schedule, n_periods, availability)

## Create readable meeting schedule


In [12]:
meeting_schedule = hb.generate_meeting_schedule(contacts, bool_schedule, folderpath=parent_fullpath, save=False)
meeting_schedule

,Period 1,Period 2,Period 3,Period 4,Period 5
Person,,,,,
Dale Frye,Andre Rios,James Edwards,Michael Carrillo,Robert Nguyen,Ann Jefferson
Ann Jefferson,Michael Carrillo,Robert Nguyen,James Edwards,Andre Rios,Dale Frye
Robert Nguyen,James Edwards,Ann Jefferson,Andre Rios,Dale Frye,Michael Carrillo
James Edwards,Robert Nguyen,Dale Frye,Ann Jefferson,Michael Carrillo,Andre Rios
Andre Rios,Dale Frye,Michael Carrillo,Robert Nguyen,Ann Jefferson,James Edwards
Michael Carrillo,Ann Jefferson,Andre Rios,Dale Frye,James Edwards,Robert Nguyen


## Creat a single period's list of meetings - ready for mail merge export

In [13]:
all_meetings_period = hb.period_meeting_list(contacts, bool_schedule, 1, full=True, save=False, folderpath=parent_fullpath)
all_meetings_period

,Person 1,Person 2,Person 1 email,Person 2 email,Person 1 assistant,Person 2 assistant,Person 1 assistant email,Person 2 assistant email,Assistant's emails
0,James Edwards,Robert Nguyen,JamesEdwards@email.com,RobertNguyen@email.com,James Edwards,Robert Nguyen,JamesEdwards@email.com,RobertNguyen@email.com,JamesEdwards@email.com; RobertNguyen@email.com
1,Andre Rios,Dale Frye,AndreRios@email.com,DaleFrye@email.com,Andre Rios,Dale Frye,AndreRios@email.com,DaleFrye@email.com,AndreRios@email.com; DaleFrye@email.com
2,Michael Carrillo,Ann Jefferson,MichaelCarrillo@email.com,AnnJefferson@email.com,Michael Carrillo,Ann Jefferson,MichaelCarrillo@email.com,AnnJefferson@email.com,MichaelCarrillo@email.com; AnnJefferson@email.com
